# How to read data

Use JumpPoseDataset in dataset.py to create a dataset that iterates over jump (xyzt) data and its label data as a tuple.

In [4]:
import sys
sys.path.append('..')
from dataset.dataset import JumpPoseDataset

In [5]:
pose_paths = '../dataset/data'
info_path = '../dataset/info.xlsx'
dataset = JumpPoseDataset(pose_paths, info_path)
pose, label = dataset.get_example(10)
print(pose.shape, label.shape)

(1005, 87) (5,)


In [59]:
import chainer

iterator = chainer.iterators.SerialIterator(dataset, len(dataset))
batch = iterator.next()
x, y = zip(*batch)
x, y = np.asarray(x), np.asarray(y)
x = x.reshape(1,-1)
y = y[:,0].reshape(1,-1)
print(x.shape, y.shape)

(1, 40) (1, 40)


# How to visualize data

Use make_video from utls to make a .mp4 of the pose data.

In [60]:
import utls
from IPython.display import Video
import importlib
importlib.reload(utls)
out = './assets/gifs/pose_example.mp4'
utls.make_video(pose, out)
Video(out)  

Traceback (most recent call last):
  File "/home/atom/anaconda3/lib/python3.7/site-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/home/atom/anaconda3/lib/python3.7/site-packages/matplotlib/animation.py", line 953, in _start
    self._init_draw()
  File "/home/atom/anaconda3/lib/python3.7/site-packages/matplotlib/animation.py", line 1732, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File "/home/atom/anaconda3/lib/python3.7/site-packages/matplotlib/animation.py", line 1755, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
  File "../utls.py", line 32, in update
    graph.set_data(pose[i,:,0],pose[i,:,2])
IndexError: too many indices for array


IndexError: too many indices for array

# Action Quality Assesment  

## Points to consider

- Long term motion dynamics (1000+ frames)
    - Attention
- Varying sequence length
    - Naive Aggregation
    - DTW
    - TE-Features
    - BiLSTM-AutoEncoding

- Small data
- Ill-defined classes

## DTW


In [61]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

def dtw_wrap(x, y): # wrap because fastdtw also outputs paths
    distance, _ = fastdtw(x, y , dist=euclidean)
    return distance

dtw_wrap(x[0], x[1])

IndexError: index 1 is out of bounds for axis 0 with size 1

In [62]:
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=1, metric=dtw_wrap)

classifier.fit(x, y)
score(classifier, x, y)

ValueError: setting an array element with a sequence.

In [16]:
import chainer 
iterator = chainer.iterators.SerialIterator(dataset, len(dataset))
batch = iterator.next()

In [7]:
batch[0][0].shape

(1212, 29, 3)

In [9]:
train, target = chainer.dataset.concat_examples(batch, -1)

ValueError: all the input array dimensions except for the concatenation axis must match exactly